In [8]:
%load_ext jupyter_black

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import os
from shapely.geometry import Polygon

os.environ["USE_PYGEOS"] = "0"
import geopandas
import shutil
import numpy as np
import napari
import os
import spatialdata as sd
from spatialdata.transformations import (
    Affine,
    Sequence,
    Identity,
    align_elements_using_landmarks,
    get_transformation,
    set_transformation,
)
from napari_spatialdata import Interactive

In [9]:
print("current working directory:", os.getcwd())
SPATIALDATA_SANDBOX_PATH = "spatialdata-sandbox"
assert os.path.isdir(
    SPATIALDATA_SANDBOX_PATH
), f"{SPATIALDATA_SANDBOX_PATH} not found, please use symlinks to make it available"
GENERATED_DATA_PATH = os.path.join(
    SPATIALDATA_SANDBOX_PATH, "generated_data/xenium_visium_integration"
)
assert os.path.isdir(
    GENERATED_DATA_PATH
), f"{GENERATED_DATA_PATH} not found, please use symlinks to make it available"


XE_REP1_PATH = os.path.join(SPATIALDATA_SANDBOX_PATH, "xenium_rep1_io/data.zarr")
XE_REP2_PATH = os.path.join(SPATIALDATA_SANDBOX_PATH, "xenium_rep2_io/data.zarr")
VISIUM_PATH = os.path.join(
    SPATIALDATA_SANDBOX_PATH, "visium_associated_xenium_io/data.zarr"
)

XE_REP1_ROI_PATH = os.path.join(GENERATED_DATA_PATH, "xe_rep1_roi.zarr")
XE_REP2_ROI_PATH = os.path.join(GENERATED_DATA_PATH, "xe_rep2_roi.zarr")
VISIUM_ROI_PATH = os.path.join(GENERATED_DATA_PATH, "visium_roi.zarr")

LANDMARKS_SDATA_PATH = os.path.join(GENERATED_DATA_PATH, "sandbox.zarr")

current working directory: /Users/macbook/embl/projects/basel/spatialdata-sandbox/notebooks


In [11]:
xe_rep1_sdata = sd.read_zarr(XE_REP1_PATH)
xe_rep2_sdata = sd.read_zarr(XE_REP2_PATH)
visium_sdata = sd.read_zarr(VISIUM_PATH)

xe_rep1_roi_sdata = sd.read_zarr(XE_REP1_ROI_PATH)
xe_rep2_roi_sdata = sd.read_zarr(XE_REP2_ROI_PATH)
visium_roi_sdata = sd.read_zarr(VISIUM_ROI_PATH)

landmarks_sdata = sd.read_zarr(LANDMARKS_SDATA_PATH)

INFO     Instance key `cell_id` could be of type `pd.Categorical`. Consider casting it.                            
INFO     Instance key `cell_id` could be of type `pd.Categorical`. Consider casting it.                            


In [20]:
shapes_full = {
    "visium_spots": visium_sdata.shapes["CytAssist_FFPE_Human_Breast_Cancer"],
    "xe_rep1_cells": xe_rep1_sdata.shapes["cell_circles"],
    "xe_rep2_cells": xe_rep2_sdata.shapes["cell_circles"],
}

shapes_roi = {
    "visium_spots": visium_roi_sdata.shapes["CytAssist_FFPE_Human_Breast_Cancer"],
    "xe_rep1_cells": xe_rep1_roi_sdata.shapes["cell_circles"],
    "xe_rep2_cells": xe_rep2_roi_sdata.shapes["cell_circles"],
}

images = {
    "xe_rep1_im": xe_rep1_sdata.images["morphology_mip"],
    "xe_rep2_im": xe_rep2_sdata.images["morphology_mip"],
    "visium_im": visium_sdata.images["CytAssist_FFPE_Human_Breast_Cancer_full_image"],
}

sdata_napari = sd.SpatialData(
    images=images,
    shapes=shapes_full | {"box": landmarks_sdata.shapes["box"]},
)

sdata_roi_napari = sd.SpatialData(
    images=images,
    shapes=shapes_roi | {"box": landmarks_sdata.shapes["box"]},
)

In [21]:
sdata_napari

SpatialData object with:
├── Images
│     ├── 'visium_im': MultiscaleSpatialImage[cyx] (3, 21571, 19505), (3, 10785, 9752), (3, 5392, 4876), (3, 2696, 2438), (3, 1348, 1219)
│     ├── 'xe_rep1_im': MultiscaleSpatialImage[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
│     └── 'xe_rep2_im': MultiscaleSpatialImage[cyx] (1, 25779, 35411), (1, 12889, 17705), (1, 6444, 8852), (1, 3222, 4426), (1, 1611, 2213)
└── Shapes
      ├── 'box': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'visium_spots': GeoDataFrame shape: (4992, 2) (2D shapes)
      ├── 'xe_rep1_cells': GeoDataFrame shape: (167780, 2) (2D shapes)
      └── 'xe_rep2_cells': GeoDataFrame shape: (118752, 2) (2D shapes)
with coordinate systems:
▸ 'downscaled_hires', with elements:
        visium_spots (Shapes)
▸ 'downscaled_lowres', with elements:
        visium_spots (Shapes)
▸ 'global', with elements:
        visium_im (Images), xe_rep1_im (Images), xe_rep2_im (Images), box (Shapes)

In [22]:
sdata_roi_napari

SpatialData object with:
├── Images
│     ├── 'visium_im': MultiscaleSpatialImage[cyx] (3, 21571, 19505), (3, 10785, 9752), (3, 5392, 4876), (3, 2696, 2438), (3, 1348, 1219)
│     ├── 'xe_rep1_im': MultiscaleSpatialImage[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
│     └── 'xe_rep2_im': MultiscaleSpatialImage[cyx] (1, 25779, 35411), (1, 12889, 17705), (1, 6444, 8852), (1, 3222, 4426), (1, 1611, 2213)
└── Shapes
      ├── 'box': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'visium_spots': GeoDataFrame shape: (2756, 2) (2D shapes)
      ├── 'xe_rep1_cells': GeoDataFrame shape: (99221, 2) (2D shapes)
      └── 'xe_rep2_cells': GeoDataFrame shape: (98365, 2) (2D shapes)
with coordinate systems:
▸ 'aligned', with elements:
        visium_spots (Shapes), xe_rep1_cells (Shapes), xe_rep2_cells (Shapes)
▸ 'global', with elements:
        visium_im (Images), xe_rep1_im (Images), xe_rep2_im (Images), box (Shapes)

In [25]:
Interactive(sdata_napari)

2023-03-24 00:18:25.024 | WARNING  | napari_spatialdata.interactive:_add_circles:255 - Too many shapes 167780 > 10000, using points instead of ellipses. Size will stop being correct beyond a certain zoom level
2023-03-24 00:18:25.137 | WARNING  | napari_spatialdata.interactive:_add_circles:255 - Too many shapes 118752 > 10000, using points instead of ellipses. Size will stop being correct beyond a certain zoom level


In [26]:
Interactive(sdata_roi_napari)

2023-03-24 00:18:47.250 | WARNING  | napari_spatialdata.interactive:_add_circles:255 - Too many shapes 99221 > 10000, using points instead of ellipses. Size will stop being correct beyond a certain zoom level
2023-03-24 00:18:47.340 | WARNING  | napari_spatialdata.interactive:_add_circles:255 - Too many shapes 98365 > 10000, using points instead of ellipses. Size will stop being correct beyond a certain zoom level
